In [1]:
import torch 
x = torch.arange(-10, 10).float() / 10
x.unsqueeze_(1)
y = x ** 3

In [9]:
from src.ptf.trainer import _BaseTrainer
from src.ptf.model import _BaseModel
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.optim import SGD

class Recon(_BaseModel):
    def compute_loss(self, network, batch):
        x, y = batch
        out = network(x)
        return nn.MSELoss()(out, y)

dataset = TensorDataset(x, y)
network = nn.Sequential(
    nn.Linear(1, 8),
    nn.ReLU(),
    nn.Linear(8, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)
model = Recon()
optimizer = SGD(network.parameters(), lr = 0.01)

trainer = _BaseTrainer(
    hparams=dict(
        epoch_duration = 40,
        batch_size = 4
    ),
    modules=dict(
        dataset=dataset,
        network=network,
        model=model,
        optimizer=optimizer
    )
)

In [3]:
len(dataset), trainer.epoch_length

(20, 5)

In [11]:
trainer.loop()

In [12]:
((network(x) - y) ** 2).sum()

tensor(0.8949, grad_fn=<SumBackward0>)